LSASS Memory Read

In [ ]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`
import $ivy.`io.delta:delta-core_2.12:0.6.1`
import $ivy.`com.lihaoyi::requests:0.8.0`
import org.apache.spark.sql._
import io.delta.sql._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")      
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
}
import spark.implicits._
import java.io.{FileInputStream, FileOutputStream}
import java.util.zip.ZipInputStream

os.write(
  os.pwd / "empire_mimikatz_logonpasswords.zip",
  requests.get.stream("https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/credential_access/host/empire_mimikatz_logonpasswords.zip")
)

val empireZip = new ZipInputStream(new FileInputStream("empire_mimikatz_logonpasswords.zip"))
val file = empireZip.getNextEntry
val fout = new FileOutputStream(file.getName)
val buffer = new Array[Byte](1024)
Stream.continually(empireZip.read(buffer)).takeWhile(_ != -1).foreach(fout.write(buffer, 0, _))

val df = spark.read.json("empire_mimikatz_logonpasswords_2020-08-07103224.json")
df.write.format("delta").mode("overwrite").save("delta_table/.")
df.count()



In [ ]:
df.head()

In [ ]:
df.select("@timestamp","Hostname","SubjectUserName","ProcessName","ObjectName","AccessMask","EventID").filter(
    ('EventID === 4663 || 'EventID == 4656)
    && 'ObjectName.like("%lsass.exe")
    && !'SubjectUserName.like("%$")
)
df.head()

In [ ]:
val processAccessDf = df.select("@timestamp","Hostname","SourceImage","TargetImage","GrantedAccess","SourceProcessGUID","CallTrace")
.filter(
    'Channel === "Microsoft-Windows-Sysmon/Operational"
    && 'EventID === 10
    && 'TargetImage.like("%lsass.exe")
    && 'CallTrace.like("%UNKNOWN%"))
df.head()

In [ ]:
val imageLoadDf = df.select("ProcessGuid","Image","ImageLoaded").filter(
    'Channel === "Microsoft-Windows-Sysmon/Operational"
    && 'EventID === 7
    && ( 
        'ImageLoaded.like("%samlib.dll") || 'ImageLoaded.like("%vaultcli.dll") || 'ImageLoaded.like("%hid.dll") || 'ImageLoaded.like("%winscard.dll") || 'ImageLoaded.like("%cryptdll.dll")
    )
)

In [ ]:
processAccessDf.join(imageLoadDf,processAccessDf("SourceProcessGUID") === imageLoadDf("ProcessGuid"),"inner").show()